In [28]:
import bs4
import csv
import datetime
import urllib.request
import json
import pymysql

# 프리미어리그 테이블 URL
Primer_Lig_URL = "https://www.premierleague.com/tables"

# 웹 페이지 요청 및 HTML 파싱
html_obj = urllib.request.urlopen(Primer_Lig_URL)
web_page = html_obj.read()
soup = bs4.BeautifulSoup(web_page, "html.parser")

# 테이블의 모든 팀 정보 가져오기
rows = soup.select("tbody.league-table__tbody tr")

# 결과 저장 리스트
teams_data = []

for row in rows:
    # 팀 이름 가져오기
    team_name_tag = row.select_one("td.league-table__team a span.long")
    if team_name_tag:
        team_name = team_name_tag.text.strip()
    else:
        continue  # 팀 이름이 없으면 건너뛴다.

    # 경기 데이터 가져오기
    stats = [td.text.strip() for td in row.select("td")[2:10]]

    # 데이터 리스트에 추가
    teams_data.append([team_name] + stats)

# 컬럼명 정의
columns = ["Club", "Played", "Won", "Drawn", "Lost", "GF", "GA", "GD", "Points"]

In [29]:
with open('premier league.json', 'w', encoding='utf-8') as outfile:
    jsonFile = json.dumps(teams_data, indent=4, sort_keys=True, ensure_ascii=False)
    outfile.write(jsonFile)

In [30]:
result_list = []
for content in teams_data:
    result_list.append(teams_data)

with open('premier league.csv', 'a', newline='') as csvfile:
    fieldnames = columns
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    for result in teams_data:
        writer.writerow(dict(zip(fieldnames, result)))

In [31]:
db = pymysql.connect(host="localhost", user="webuser", passwd="webuser", db="webdb")

cursor = db.cursor()

query = "INSERT INTO premierleague (Club, Played, Won, Drawn, Lost, GF, GA, GD, Points) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    cursor.executemany(query, teams_data)
    db.commit()
    print(f"DB 저장 완료")
except Exception as e:
    db.rollback()
    print(f"DB 저장 실패 : {e}")
cursor.close()
db.close()

DB 저장 완료
